<a href="https://colab.research.google.com/github/GirumSe/iCog-Internship-Tasks/blob/main/RAG_Task1_by_Girum_Senay.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Installing and Preparing the Packages needed

In [ ]:
!pip install accelerate langchain bitsandbytes -qq
!pip install pinecone-client -qq
!pip install llama_index sentence_transformers -qq
!pip install langchain-community -qq
!pip install together -qq
!pip install neo4j -qq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 390.9 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 997.8/997.8 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 391.5/391.5 kB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.4/140.4 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.8/244.8 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.6/117.6 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.5/15.5 MB 59.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 52.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 169.4/169.4 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from google.colab import userdata
uri = "neo4j+s://af34469f.databases.neo4j.io"
user = "neo4j"
password = "xtAnOiQRD_V1jgk4kJkpSgOdXuJ9lvKGguKm9dJIQUU"
together_key = "69bff935fefb1e49af68b1ec6b8b01c94f184d318baffbe3e8e83866a0c11587"
pinecone_key= "1d454642-7172-488f-85ec-aac424aa98c5"

#Neo4j

##Querying

In [ ]:
from neo4j import GraphDatabase
driver = GraphDatabase.driver(uri, auth=(user, password))

###Query and Other utility Function

In [ ]:
def run_query(query, parameters=None):
    with driver.session() as session:
        results = session.run(query, parameters)
        return [dict(record) for record in results]

In [ ]:
# query = "MATCH (n) RETURN n"
query = """
MATCH (n)-[r]->(m)
RETURN n, r, m
LIMIT 25
"""

result = run_query(query)
print(result)

[{'n': <Node element_id='4:613bff9a-aaea-44be-a38a-301839ff4e72:0' labels=frozenset({'PLAYER'}) properties={'number': 0, 'name': 'Russell Westbrook', 'weight': 91, 'age': 33, 'height': 1.91}>, 'r': <Relationship element_id='5:613bff9a-aaea-44be-a38a-301839ff4e72:6919780827454767105' nodes=(<Node element_id='4:613bff9a-aaea-44be-a38a-301839ff4e72:0' labels=frozenset({'PLAYER'}) properties={'number': 0, 'name': 'Russell Westbrook', 'weight': 91, 'age': 33, 'height': 1.91}>, <Node element_id='4:613bff9a-aaea-44be-a38a-301839ff4e72:1' labels=frozenset({'PLAYER'}) properties={'number': 6, 'name': 'LeBron James', 'weight': 113, 'age': 36, 'height': 2.06}>) type='TEAMMATES' properties={}>, 'm': <Node element_id='4:613bff9a-aaea-44be-a38a-301839ff4e72:1' labels=frozenset({'PLAYER'}) properties={'number': 6, 'name': 'LeBron James', 'weight': 113, 'age': 36, 'height': 2.06}>}, {'n': <Node element_id='4:613bff9a-aaea-44be-a38a-301839ff4e72:0' labels=frozenset({'PLAYER'}) properties={'number': 0, 

###Query The Database Using Natural Language

In [ ]:
from together import Together
import re
client = Together(api_key=together_key)

In [ ]:
prompt = """give me a cypher query for a neo4j database for the following natural language query,
use TEAM, COACH, and PLAYER in capital letters FOR NODES,
PLAYERs have properties name, age, weight, height,
COACHs COACHES_FOR TEAMs and PLAYERs PLAYS_FOR TEAMs,
return only the query in square brackets and don't comment on anything else,
and use arrows pointing in the direction of the TEAM,
their is PLAYERs' statistics from PLAYED_AGAINST relationships in format `(lebron)-[:PLAYED_AGAINST {minutes: 38, points: 32, assists: 6, rebounds: 6, turnovers: 2}]-> (memphis)`
and salary from PLAYS_FOR in format `(lebron)-[:PLAYS_FOR {salary: 40000000}]-> (lakers)` relationships.
"""
prompt2 = "return only the query in square brackets and don't format it and don't comment on anything else,!"
stream = client.chat.completions.create(
model="meta-llama/Meta-Llama-3.1-70B-Instruct-Turbo",
messages=[{"role": "user", "content": prompt + """return list of all players with thier properties using symbol p dont give the aliases, use like p.name, p.age, p.weight, p.height
 and their salary with symbol pf no stastics, """ + prompt2}],
stream=True,
)
query = ""
for chunk in stream:
  query += chunk.choices[0].delta.content or ""
  print(chunk.choices[0].delta.content or "", end="", flush=True)

clean_text = re.sub(r'^\[|\]$', '', query)
print('\ncleaned text\n',clean_text)
result = run_query(clean_text)
print('\nresult\n',result)

[MATCH (p:PLAYER)-[pf:PLAYS_FOR]->(t:TEAM) RETURN p.name, p.age, p.weight, p.height, pf.salary]
cleaned text
 MATCH (p:PLAYER)-[pf:PLAYS_FOR]->(t:TEAM) RETURN p.name, p.age, p.weight, p.height, pf.salary

result
 [{'p.name': 'Russell Westbrook', 'p.age': 33, 'p.weight': 91, 'p.height': 1.91, 'pf.salary': 33000000}, {'p.name': 'LeBron James', 'p.age': 36, 'p.weight': 113, 'p.height': 2.06, 'pf.salary': 40000000}, {'p.name': 'Anthony Davis', 'p.age': 28, 'p.weight': 115, 'p.height': 2.08, 'pf.salary': 38000000}, {'p.name': 'Ja Morant', 'p.age': 22, 'p.weight': 79, 'p.height': 1.91, 'pf.salary': 8000000}, {'p.name': 'Luka Doncic', 'p.age': 22, 'p.weight': 104, 'p.height': 2.01, 'pf.salary': 50000000}, {'p.name': 'Kristaps Porzingis', 'p.age': 26, 'p.weight': 109, 'p.height': 2.21, 'pf.salary': 26000000}, {'p.name': 'Kevin Durant', 'p.age': 33, 'p.weight': 109, 'p.height': 2.08, 'pf.salary': 45000000}, {'p.name': 'James Harden', 'p.age': 32, 'p.weight': 100, 'p.height': 1.96, 'pf.salary': 

In [ ]:
descriptions = []

# Convert to natural language descriptions and save to the list
for record in result:
    name = record['p.name']
    age = record['p.age']
    weight = record['p.weight']
    height = record['p.height']
    salary = record['pf.salary']

    description = f"Player {name}, whose age is {age}, weighs {weight} pounds and has a height of {height} meters. his salary is {salary}."
    descriptions.append(description)

In [ ]:
descriptions

['Player Russell Westbrook, whose age is 33, weighs 91 pounds and has a height of 1.91 meters. his salary is 33000000.',
 'Player LeBron James, whose age is 36, weighs 113 pounds and has a height of 2.06 meters. his salary is 40000000.',
 'Player Anthony Davis, whose age is 28, weighs 115 pounds and has a height of 2.08 meters. his salary is 38000000.',
 'Player Ja Morant, whose age is 22, weighs 79 pounds and has a height of 1.91 meters. his salary is 8000000.',
 'Player Luka Doncic, whose age is 22, weighs 104 pounds and has a height of 2.01 meters. his salary is 50000000.',
 'Player Kristaps Porzingis, whose age is 26, weighs 109 pounds and has a height of 2.21 meters. his salary is 26000000.',
 'Player Kevin Durant, whose age is 33, weighs 109 pounds and has a height of 2.08 meters. his salary is 45000000.',
 'Player James Harden, whose age is 32, weighs 100 pounds and has a height of 1.96 meters. his salary is 4200000.',
 'Player Giannis Antetokounmpo, whose age is 26, weighs 110 

In [ ]:
prompt = """give me a cypher query for a neo4j database for the following natural language query,
use TEAM, COACH, and PLAYER in capital letters FOR NODES,
PLAYERs have properties name, age, weight, height,
COACHs COACHES_FOR TEAMs and PLAYERs PLAYS_FOR TEAMs,
return only the query in square brackets and don't comment on anything else,
and use arrows pointing in the direction of the TEAM,
their is PLAYERs' statistics from PLAYED_AGAINST relationships in format `(lebron)-[:PLAYED_AGAINST {minutes: 38, points: 32, assists: 6, rebounds: 6, turnovers: 2}]-> (memphis)`
and salary from PLAYS_FOR in format `(lebron)-[:PLAYS_FOR {salary: 40000000}]-> (lakers)` relationships.
"""
prompt2 = "return only the query in square brackets and don't format it and don't comment on anything else,!"
stream = client.chat.completions.create(
    model="meta-llama/Meta-Llama-3.1-70B-Instruct-Turbo",
    messages=[{"role": "user", "content": prompt + " return the name coches and players for each team, when you return use the nameing t.name as Team, Coaches as Coaches, Players as players " + prompt2}],
    stream=True,
)
query = ""
for chunk in stream:
    query += chunk.choices[0].delta.content or ""
    print(chunk.choices[0].delta.content or "", end="", flush=True)

clean_text = re.sub(r'^\[|\]$', '', query)
print('\ncleaned text\n', clean_text)
result = run_query(clean_text)
print('result\n', result)

[MATCH (t:TEAM)<-[:COACHES_FOR]-(c:COACH), (t:TEAM)<-[:PLAYS_FOR]-(p:PLAYER) RETURN t.name AS Team, collect(c.name) AS Coaches, collect(p.name) AS Players]
cleaned text
 MATCH (t:TEAM)<-[:COACHES_FOR]-(c:COACH), (t:TEAM)<-[:PLAYS_FOR]-(p:PLAYER) RETURN t.name AS Team, collect(c.name) AS Coaches, collect(p.name) AS Players
result
 [{'Team': 'LA Lakers', 'Coaches': ['Frank Vogel', 'Frank Vogel', 'Frank Vogel'], 'Players': ['Russell Westbrook', 'LeBron James', 'Anthony Davis']}, {'Team': 'Memphis Grizzlies', 'Coaches': ['Taylor Jenkins'], 'Players': ['Ja Morant']}, {'Team': 'Dallas Mavericks', 'Coaches': ['Jason Kidd', 'Jason Kidd'], 'Players': ['Luka Doncic', 'Kristaps Porzingis']}, {'Team': 'Brooklyn Nets', 'Coaches': ['Steve Nash', 'Steve Nash'], 'Players': ['Kevin Durant', 'James Harden']}, {'Team': 'Milwaukee Bucks', 'Coaches': ['Mike Budenholzer', 'Mike Budenholzer'], 'Players': ['Giannis Antetokounmpo', 'Khris Middleton']}, {'Team': 'Philadelphia 76ers', 'Coaches': ['Doc Rivers', '

In [ ]:
formatted_teams = []
for team in result:
    team_name = team['Team']
    coaches = ', '.join(set(team['Coaches']))
    players = ', '.join(team['Players'])
    statement = f"{team_name} is coached by {coaches}. Players who play for {team_name} are {players}."
    formatted_teams.append(statement)

In [ ]:
formatted_teams

['LA Lakers is coached by Frank Vogel. Players who play for LA Lakers are Russell Westbrook, LeBron James, Anthony Davis.',
 'Memphis Grizzlies is coached by Taylor Jenkins. Players who play for Memphis Grizzlies are Ja Morant.',
 'Dallas Mavericks is coached by Jason Kidd. Players who play for Dallas Mavericks are Luka Doncic, Kristaps Porzingis.',
 'Brooklyn Nets is coached by Steve Nash. Players who play for Brooklyn Nets are Kevin Durant, James Harden.',
 'Milwaukee Bucks is coached by Mike Budenholzer. Players who play for Milwaukee Bucks are Giannis Antetokounmpo, Khris Middleton.',
 'Philadelphia 76ers is coached by Doc Rivers. Players who play for Philadelphia 76ers are Joel Embiid, Tobias Harris.']

In [ ]:
prompt = """
Generate a Cypher query for the following:
- Use TEAM, COACH, and PLAYER in uppercase for nodes.
- PLAYERs have properties: name, age, weight, height.
- Relationships: PLAYERs PLAYS_FOR TEAMs, COACHs COACHES_FOR TEAMs, PLAYERs have statistics from PLAYED_AGAINST.
- Format PLAYED_AGAINST statistics as: (player)-[:PLAYED_AGAINST {minutes: x, points: y, assists: z, rebounds: a, turnovers: b}]->(team).
- Format PLAYS_FOR salary as: (player)-[:PLAYS_FOR {salary: s}]->(team).
- Return only the Cypher query, without any comments or extra formatting.

Query requirements:
- Retrieve each PLAYER's name, individual game stats (match_stats), and total stats (total_stat).
- The query should return results :
  - `Name` as the player's name.
  - `match_stats` as a list of maps, with each map containing `against`, `points`, `assists`, `rebounds`, `turnovers`, and `minutes`.
  - `total_stat` as a map containing the total points, assists, rebounds, turnovers, and minutes.
- Ensure the query uses a `WITH` clause to properly aggregate statistics and group data before returning it.
- Use the `WITH` clause to alias the player's name and aggregate statistics.
- use sum in r.[stat_properties name] when calculating the total.
- The final Cypher query should be valid, executable, and avoid any aggregation errors.
"""

prompt2 = " return only the query in square brackets and don't format it and don't comment on anything else,!"
stream = client.chat.completions.create(
    model="meta-llama/Meta-Llama-3.1-70B-Instruct-Turbo",
    messages=[{"role": "user", "content": prompt + prompt2}],
    stream=True,
)
query = ""
for chunk in stream:
    query += chunk.choices[0].delta.content or ""
    print(chunk.choices[0].delta.content or "", end="", flush=True)

[
MATCH (p:PLAYER)-[r:PLAYED_AGAINST]->(t:TEAM)
WITH p.name AS Name, collect({against: t.name, points: r.points, assists: r.assists, rebounds: r.rebounds, turnovers: r.turnovers, minutes: r.minutes}) AS match_stats, 
sum(r.points) AS total_points, sum(r.assists) AS total_assists, sum(r.rebounds) AS total_rebounds, sum(r.turnovers) AS total_turnovers, sum(r.minutes) AS total_minutes
RETURN Name, match_stats, {points: total_points, assists: total_assists, rebounds: total_rebounds, turnovers: total_turnovers, minutes: total_minutes} AS total_stat
]

In [ ]:
clean_text = re.sub(r'^\[|\]$', '', query)
print('\ncleaned text\n', clean_text)
result = run_query(clean_text)
print('result\n', result)


cleaned text
 
MATCH (p:PLAYER)-[r:PLAYED_AGAINST]->(t:TEAM)
WITH p.name AS Name, collect({against: t.name, points: r.points, assists: r.assists, rebounds: r.rebounds, turnovers: r.turnovers, minutes: r.minutes}) AS match_stats, 
sum(r.points) AS total_points, sum(r.assists) AS total_assists, sum(r.rebounds) AS total_rebounds, sum(r.turnovers) AS total_turnovers, sum(r.minutes) AS total_minutes
RETURN Name, match_stats, {points: total_points, assists: total_assists, rebounds: total_rebounds, turnovers: total_turnovers, minutes: total_minutes} AS total_stat

result
 [{'Name': 'Ja Morant', 'match_stats': [{'assists': 7, 'rebounds': 8, 'minutes': 43, 'against': 'LA Lakers', 'turnovers': 4, 'points': 42}, {'assists': 3, 'rebounds': 4, 'minutes': 35, 'against': 'LA Lakers', 'turnovers': 2, 'points': 35}, {'assists': 13, 'rebounds': 6, 'minutes': 26, 'against': 'Brooklyn Nets', 'turnovers': 2, 'points': 32}], 'total_stat': {'assists': 23, 'rebounds': 18, 'minutes': 104, 'turnovers': 8, 'poi

In [ ]:
sentences = []

for player in result:
    name = player['Name']
    total_stat = player['total_stat']
    match_stats = player['match_stats']

    for match in match_stats:
        sentence = (f"{name} played against {match['against']} and scored {match['points']} points, "
                    f"with {match['assists']} assists, {match['rebounds']} rebounds, "
                    f"and {match['turnovers']} turnovers in {match['minutes']} minutes.")
        sentences.append(sentence)

    total_sentence = (f"{name} totaled {total_stat['points']} points, {total_stat['assists']} assists, "
                      f"{total_stat['rebounds']} rebounds, {total_stat['turnovers']} turnovers, "
                      f"and played for {total_stat['minutes']} minutes across all games.")
    sentences.append(total_sentence)

In [ ]:
sentences

['Ja Morant played against LA Lakers and scored 42 points, with 7 assists, 8 rebounds, and 4 turnovers in 43 minutes.',
 'Ja Morant played against LA Lakers and scored 35 points, with 3 assists, 4 rebounds, and 2 turnovers in 35 minutes.',
 'Ja Morant played against Brooklyn Nets and scored 32 points, with 13 assists, 6 rebounds, and 2 turnovers in 26 minutes.',
 'Ja Morant totaled 109 points, 23 assists, 18 rebounds, 8 turnovers, and played for 104 minutes across all games.',
 'Kevin Durant played against LA Lakers and scored 45 points, with 5 assists, 8 rebounds, and 2 turnovers in 43 minutes.',
 'Kevin Durant played against Memphis Grizzlies and scored 37 points, with 2 assists, 12 rebounds, and 1 turnovers in 34 minutes.',
 'Kevin Durant played against Dallas Mavericks and scored 28 points, with 6 assists, 8 rebounds, and 0 turnovers in 29 minutes.',
 'Kevin Durant totaled 110 points, 13 assists, 28 rebounds, 3 turnovers, and played for 106 minutes across all games.',
 'James Harde

##Adding to the Pincone Database

###preparing embaddings

In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings, SentenceTransformerEmbeddings
from llama_index.legacy.embeddings.langchain import LangchainEmbedding

embed_model = LangchainEmbedding(
  HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")
)

[nltk_data] Downloading package stopwords to
[nltk_data]     /usr/local/lib/python3.10/dist-
[nltk_data]     packages/llama_index/legacy/_static/nltk_cache...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to
[nltk_data]     /usr/local/lib/python3.10/dist-
[nltk_data]     packages/llama_index/legacy/_static/nltk_cache...
[nltk_data]   Unzipping tokenizers/punkt.zip.
/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:141: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.auto

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
facts = formatted_teams + descriptions + sentences

In [ ]:
embeddings = []

for i in facts:
    section_embedding = embed_model.get_text_embedding(i)
    embeddings.append(section_embedding)
    print(section_embedding)

[0.0497715137898922, -0.02506810799241066, -0.016862777993083, 0.02750435099005699, 0.00921717844903469, 0.0013143144315108657, -0.010095849633216858, 0.0007294078823179007, 0.009697581641376019, -0.032910171896219254, -0.02864459529519081, 0.00646508764475584, 0.017991457134485245, -0.07540023326873779, -0.002080362755805254, 0.00544921588152647, -0.0015607705572620034, 0.003174019744619727, 0.04129930958151817, -0.03133430704474449, -0.02595062181353569, 0.032570287585258484, 0.002755501540377736, 0.007992507889866829, -0.009121865034103394, -0.010524420067667961, 1.619629438209813e-05, 0.03738902509212494, 0.014421979896724224, -0.00034804869210347533, -0.04439432919025421, 0.0010071051074191928, -0.02810489386320114, -0.12010108679533005, 1.5240556194839883e-06, 0.0022433041594922543, 0.015625806525349617, -0.04496833309531212, 0.048865530639886856, 0.0055466932244598866, 0.0008097951649688184, 0.022257447242736816, 0.017077624797821045, -0.03763916715979576, 0.018871072679758072, 

### Connect to Pinecone to store the vectors

In [ ]:
from pinecone import Pinecone, ServerlessSpec
pinecone = Pinecone(
        api_key=pinecone_key)

In [ ]:
index_name = 'basketfacts'

In [ ]:
pinecone_index = pinecone.Index(index_name)
docs = []
for idx, i in enumerate(facts):
  docs.append((
        str(idx),
        embeddings[idx],
        {'text': i},
    ))
pinecone_index.upsert(vectors= docs, show_progress = True)

{'upserted_count': 67}

In [ ]:
def get_data_from_db(query):
    # embed query
    xq = embed_model.get_text_embedding(query)
    # query database
    result = pinecone_index.query(vector=xq, top_k=20, includeMetadata=True)
    matches = []
    for i in result['matches']:
      if (i['score'] > 0.3):
        # append best results
        matches.append(i.metadata['text'])
    return str(matches)

In [ ]:
DEFAULT_SYSTEM_PROMPT = """\nGENERAL INSTRUCTIONS\n
  You will be given fact sentence about the last seson's basketball facts, you will base your answares on the data you are given.
  If there isn't enough context, simply reply "i don't have enough data to tell you that\n"
  """
def get_prompt(instruction):
    prompt_template =  DEFAULT_SYSTEM_PROMPT + instruction + "\n"
    return prompt_template

In [ ]:
def format_prompt(query, context):
    return '''
    ### data:
    {context}
    based on this data
    ### User Question:
    {query}
    '''.format(context=context, query=query)

In [ ]:
def answer(query):
    context = get_data_from_db(query)
    prompt = format_prompt(query, context)
    prompt_template = get_prompt(prompt)
    stream = client.chat.completions.create(
    model="meta-llama/Meta-Llama-3.1-70B-Instruct-Turbo",
    messages=[{"role": "user", "content": prompt_template}],
    stream=True,
    )

    for chunk in stream:
        print(chunk.choices[0].delta.content or "", end="", flush=True)

##Prompting

Prompts that i tested are:

*   "who played the most minutes"
*   "which player have the most assists in a single game?"
*   "Who coaches Dallas Mavericks?"

In [ ]:
answer("Who coaches Dallas Mavericks?")

Jason Kidd